In [1]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras import layers, activations, models, preprocessing

In [2]:
!pip install kaggle --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"thingcph","key":"874752620ff669e2fda909690de005fd"}'}

In [4]:
!mkdir -p ~/.kaggle

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
!ls ~/.kaggle

kaggle.json


In [7]:
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
from tensorflow.keras import preprocessing, utils
import os
import pandas as pd

In [9]:
import re
import pandas as pd
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from keras.layers import Input,Embedding,Bidirectional,LSTM,Dense,Concatenate
from keras.models import Model
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:


questions=list()
answers=list()

filename = 'DataChatbot.csv'
df = pd.read_csv(filename,encoding = 'utf8')
df = df.astype(str)

data = df.values #numpy 
print(data.shape)
questions = data[:,1]# X
answers = data[:,-1]# y

print(questions[:10])
print(answers[:10])

(5900, 3)
['Thích mẫu người nào' 'Có crush ai không' 'Tại sao lại thích bạn dó'
 'Có hay nói chuyện không' 'Bạn kia có bắt chuyện trước không'
 'Có định rủ bạn đó về quê dịp Tết không' 'Có hay đi chơi với nhau không'
 'Sở thích mày sao?' 'Thích phim gì?' 'Thích chơi gì?']
['Dễ thương, tóc dài, da trắng' 'Có 1 bạn cùng lớp'
 'Vì đáp ứng những yêu cầu của tao' 'Hay nhắn tin messenger' 'Có đôi khi'
 'Có, đang đợi trả lời' 'Thường đi cùng cả nhóm bạn'
 'Thích phim, đồ ăn, thể thao' 'Phim ma' 'chơi đá banh']


In [12]:

import glob
import os
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 3098


In [13]:
from gensim.models import Word2Vec
import re

In [14]:
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

def tokenize(sentences):
  tokens_list = []
  vocabulary = []
  for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    tokens = sentence.split()
    vocabulary += tokens
    tokens_list.append(tokens)
  return tokens_list, vocabulary

In [15]:
#encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

(5900, 93) 93


In [16]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

(5900, 62) 62


In [17]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(5900, 62, 3098)


In [18]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 93)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 62)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 93, 200)      619600      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 62, 200)      619600      ['input_2[0][0]']                
                                                                                              

In [19]:

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=100 ) 
model.save( 'model.h5' )

Epoch 1/100
118/118 [==============================] - 117s 931ms/step - loss: 0.4134
Epoch 2/100
118/118 [==============================] - 108s 915ms/step - loss: 0.3623
Epoch 3/100
118/118 [==============================] - 108s 917ms/step - loss: 0.3519
Epoch 4/100
118/118 [==============================] - 109s 922ms/step - loss: 0.3413
Epoch 5/100
118/118 [==============================] - 108s 918ms/step - loss: 0.3305
Epoch 6/100
118/118 [==============================] - 108s 917ms/step - loss: 0.3201
Epoch 7/100
118/118 [==============================] - 109s 926ms/step - loss: 0.3104
Epoch 8/100
118/118 [==============================] - 109s 927ms/step - loss: 0.3012
Epoch 9/100
118/118 [==============================] - 109s 921ms/step - loss: 0.2921
Epoch 10/100
118/118 [==============================] - 108s 918ms/step - loss: 0.2834
Epoch 11/100
118/118 [==============================] - 108s 919ms/step - loss: 0.2750
Epoch 12/100
118/118 [==============================

In [20]:
new_model = tf.keras.models.load_model('model.h5')

new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 93)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 62)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 93, 200)      619600      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 62, 200)      619600      ['input_2[0][0]']                
                                                                                              

In [22]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [23]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : bạn có crush ai không
 có end
Enter question : bạn thích người như thế nào
 mình thích người hiền lành end
Enter question : bạn thích xem phim gì
 18 end
Enter question : bạn thích thể loại phim gì
 kinh dị end
Enter question : bạn ghét thể loại phim gì
 hành động kinh dị end
